<a href="https://colab.research.google.com/github/Srdcf/HAR/blob/main/NNModel/modeloHAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# import required libraries
import pandas as pd
import numpy as np
import seaborn as sns

import torch

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split



In [5]:
#df = pd.read_csv('/content/drive/MyDrive/NNModel/dataset1.csv')
df = pd.read_csv('/content/drive/MyDrive/NNModel/dataset2.csv')


In [6]:
df

,Unnamed: 0.1,Unnamed: 0,Latitude Change,Longitude Change,Altitude Change,Horizontal Accuracy Change,Speed Change,Gyroscope X,Gyroscope Y,Gyroscope Z,Acceleration X Change,Acceleration Y Change,Acceleration Z Change,Roll (º),Pitch (º),Yaw (º),Magnetometer X Change,Magnetometer Y Change,Magnetometer Z Change,DataLabel
0,0,0,0.0,0.0,0.0,0.0,0.0,-0.063781,0.059786,0.006924,0.000000,0.000000,0.000000,-19.75767,-11.26569,0.944726,0.000000,0.00000,0.00000,idle
1,1,1,0.0,0.0,0.0,0.0,0.0,-0.042476,0.115178,0.101730,0.000000,0.000000,0.000000,-19.66945,-11.03860,0.851585,0.000000,0.00000,0.00000,idle
2,2,2,0.0,0.0,0.0,0.0,0.0,0.053395,0.148067,0.123034,0.000000,0.000000,0.000000,-19.68774,-10.73308,0.631760,0.000000,0.00000,0.00000,idle
3,3,3,0.0,0.0,0.0,0.0,0.0,-0.029693,0.114113,0.088947,0.000000,0.000000,0.000000,-19.80198,-10.73826,0.417986,0.000000,0.00000,0.00000,idle
4,4,4,0.0,0.0,0.0,0.0,0.0,-0.072303,0.098135,0.113447,0.000000,0.000000,0.000000,-19.72018,-10.74029,0.188188,0.000000,0.00000,0.00000,idle
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119860,119860,119860,0.0,0.0,0.0,0.0,0.0,-0.023435,-0.042742,0.042876,0.172020,0.885110,-0.186050,0.00000,0.00000,0.000000,-1.199999,-0.88125,0.80625,idle
119861,119861,119861,0.0,0.0,0.0,0.0,0.0,-0.056324,-0.040612,-0.006125,0.453840,0.153720,-0.135421,0.00000,0.00000,0.000000,-1.331250,-0.95625,0.91875,idle
119862,119862,119862,0.0,0.0,0.0,0.0,0.0,-0.037150,0.020106,-0.095605,-0.634095,-0.175680,-0.034160,0.00000,0.00000,0.000000,-0.843749,-1.01250,0.50625,idle
119863,119863,119863,0.0,0.0,0.0,0.0,0.0,0.005326,0.062716,-0.018908,-0.293410,-0.297985,-0.127490,0.00000,0.00000,0.000000,-0.450000,-0.76875,0.31875,idle


In [21]:
# Encode categorical labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['DataLabelEncoder'] = label_encoder.fit_transform(df['DataLabel'])

In [22]:
# Get the mapping of original classes to their corresponding numerical values
class_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))

print("Class to Label mapping:")
print(class_mapping)


Class to Label mapping:
{'Moving': np.int64(0), 'idle': np.int64(1), 'picking': np.int64(2)}


In [23]:

# Split the data into features and target
X = (df.loc[:, 'Latitude Change':'Speed Change']).join(df.loc[:, 'Gyroscope X':'Gyroscope Z']).join(df.loc[:, 'Acceleration X Change':'Acceleration Z Change']).join(df.loc[:, 'Roll (º)':'Yaw (º)']).join(df.loc[:, 'Magnetometer X Change':'Magnetometer Z Change'])

y = df['DataLabelEncoder']  # Target

In [24]:
X


,Latitude Change,Longitude Change,Altitude Change,Horizontal Accuracy Change,Speed Change,Gyroscope X,Gyroscope Y,Gyroscope Z,Acceleration X Change,Acceleration Y Change,Acceleration Z Change,Roll (º),Pitch (º),Yaw (º),Magnetometer X Change,Magnetometer Y Change,Magnetometer Z Change
0,0.0,0.0,0.0,0.0,0.0,-0.063781,0.059786,0.006924,0.000000,0.000000,0.000000,-19.75767,-11.26569,0.944726,0.000000,0.00000,0.00000
1,0.0,0.0,0.0,0.0,0.0,-0.042476,0.115178,0.101730,0.000000,0.000000,0.000000,-19.66945,-11.03860,0.851585,0.000000,0.00000,0.00000
2,0.0,0.0,0.0,0.0,0.0,0.053395,0.148067,0.123034,0.000000,0.000000,0.000000,-19.68774,-10.73308,0.631760,0.000000,0.00000,0.00000
3,0.0,0.0,0.0,0.0,0.0,-0.029693,0.114113,0.088947,0.000000,0.000000,0.000000,-19.80198,-10.73826,0.417986,0.000000,0.00000,0.00000
4,0.0,0.0,0.0,0.0,0.0,-0.072303,0.098135,0.113447,0.000000,0.000000,0.000000,-19.72018,-10.74029,0.188188,0.000000,0.00000,0.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119860,0.0,0.0,0.0,0.0,0.0,-0.023435,-0.042742,0.042876,0.172020,0.885110,-0.186050,0.00000,0.00000,0.000000,-1.199999,-0.88125,0.80625
119861,0.0,0.0,0.0,0.0,0.0,-0.056324,-0.040612,-0.006125,0.453840,0.153720,-0.135421,0.00000,0.00000,0.000000,-1.331250,-0.95625,0.91875
119862,0.0,0.0,0.0,0.0,0.0,-0.037150,0.020106,-0.095605,-0.634095,-0.175680,-0.034160,0.00000,0.00000,0.000000,-0.843749,-1.01250,0.50625
119863,0.0,0.0,0.0,0.0,0.0,0.005326,0.062716,-0.018908,-0.293410,-0.297985,-0.127490,0.00000,0.00000,0.000000,-0.450000,-0.76875,0.31875


In [25]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [26]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
input_shape = X_train.shape[1]
model = models.Sequential([
    layers.Dense(512, activation='relu', input_shape=(input_shape,)),
    layers.Dropout(0.4),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(3, activation='softmax')
])

initial_learning_rate = 0.001
opt = tf.keras.optimizers.Adam(learning_rate=initial_learning_rate)
def lr_schedule(epoch):
    return initial_learning_rate * 0.95 ** epoch

lr_scheduler = LearningRateScheduler(lr_schedule)

model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=70, batch_size=128, callbacks=[lr_scheduler])

Epoch 1/70


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


656/656 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - accuracy: 0.4636 - loss: 2.3852 - learning_rate: 0.0010
Epoch 2/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.6019 - loss: 0.8210 - learning_rate: 9.5000e-04
Epoch 3/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 11s 11ms/step - accuracy: 0.6165 - loss: 0.7758 - learning_rate: 9.0250e-04
Epoch 4/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 11s 13ms/step - accuracy: 0.6166 - loss: 0.7590 - learning_rate: 8.5737e-04
Epoch 5/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.6248 - loss: 0.7454 - learning_rate: 8.1451e-04
Epoch 6/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6307 - loss: 0.7329 - learning_rate: 7.7378e-04
Epoch 7/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6361 - loss: 0.7285 - learning_rate: 7.3509e-04
Epoch 8/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 8s 12ms/step - accuracy: 0.6422 - loss: 0.7125 - learning_rate: 6.9834e-04
Epoch 9/70
656/656 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.6441 - loss: 0.7149 - lea

In [27]:
model.evaluate(X_test, y_test, batch_size=128, verbose=1)

281/281 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6812 - loss: 0.6439


[0.6395301818847656, 0.6830922961235046]

In [28]:
import numpy as np
from sklearn.metrics import confusion_matrix

# Get predictions from the model
y_pred = np.argmax(model.predict(X_test), axis=1)

# Create confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)


1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step
Confusion Matrix:
[[15665    98  1840]
 [ 2205  5036    90]
 [ 7160     3  3863]]


In [29]:
import numpy as np

# Matriz de confusão original
confusion_matrix = np.array([
    [38696, 50, 6614],
    [5588, 2257, 760],
    [18595, 25, 14315]
])

# Calcular a soma de cada linha (total de amostras por classe real)
row_sums = confusion_matrix.sum(axis=1, keepdims=True)

# Calcular a matriz de confusão normalizada por linha (em porcentagens)
confusion_matrix_percent = (confusion_matrix / row_sums) * 100

# Arredondar os valores para 2 casas decimais
confusion_matrix_percent_rounded = np.round(confusion_matrix_percent, 2)

# Exibir a matriz resultante
print("Matriz de Confusão Normalizada por Classe (em %):")
print(confusion_matrix_percent_rounded)


Matriz de Confusão Normalizada por Classe (em %):
[[8.531e+01 1.100e-01 1.458e+01]
 [6.494e+01 2.623e+01 8.830e+00]
 [5.646e+01 8.000e-02 4.346e+01]]


In [ ]:
model.save('harModel.h5')